<a href="https://colab.research.google.com/github/kikiru328/LiveStock_Spark/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### moduel import

In [115]:
import os
from glob import glob
import json
from tqdm import tqdm

import shutil

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
import cv2

# Json Test

In [75]:
json_path = '/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox'
labels = '/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox/livestock_cow_bbox_012159.json'
print(labels)
imgs = '/content/drive/MyDrive/Innopolis_spark/data/train/img/소_bbox/livestock_cow_bbox_012159.jpg'
print(imgs)

/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox/livestock_cow_bbox_012159.json
/content/drive/MyDrive/Innopolis_spark/data/train/img/소_bbox/livestock_cow_bbox_012159.jpg


In [76]:
with open(labels,'r') as json_file:
    test_j = json.load(json_file)

In [77]:
len(test_j['label_info']['annotations'])
test_j['label_info']['annotations']

[{'bbox': [1143, 486, 1319, 681], 'category_id': 1},
 {'bbox': [1241, 372, 1455, 541], 'category_id': 1},
 {'bbox': [1694, 484, 1911, 673], 'category_id': 1},
 {'bbox': [621, 418, 894, 595], 'category_id': 1},
 {'bbox': [528, 431, 653, 629], 'category_id': 1}]

In [78]:
bboxs = []
for i in tqdm(range(len(test_j['label_info']['annotations']))):
    bboxs.append(test_j['label_info']['annotations'][i]['bbox'])
    

100%|██████████| 5/5 [00:00<00:00, 16565.18it/s]


In [79]:
bboxs

[[1143, 486, 1319, 681],
 [1241, 372, 1455, 541],
 [1694, 484, 1911, 673],
 [621, 418, 894, 595],
 [528, 431, 653, 629]]

In [80]:
L = [labels]
I = [imgs]

# Remove background FC

In [81]:
# def rv_background(image,label):
#     for im,lab in tqdm(zip(image, label)):

#         img_ = cv2.imread(im)
#         img = cv2.cvtColor(img_, cv2.COLOR_BGR2RGB)

#         zero_img = np.zeros((img.shape[:2]),np.uint8)
#         with open(lab,'r') as json_file:
#             test_j = json.load(json_file)
#             coords = test_j['label_info']['annotations']
#             Bbox = []
#             for i in range(len(coords)):
#                 Bbox.append(coords[i]['bbox'])

#         for i in range(len(Bbox)):
#             f_point = Bbox[i][0], Bbox[i][1]
#             s_point = Bbox[i][2], Bbox[i][3]
#             mask = cv2.rectangle(zero_img, f_point, s_point, (255,0,0) ,-1)

#         result = cv2.bitwise_and(img,img,mask=mask)
#         return result

# Extraction_crop FC

In [82]:
def extraction_crop(image,label,folder,animal):
    for im,lab in tqdm(zip(image,label)):
        img = Image.open(im)

    with open(lab,'r') as json_file:
        test_j = json.load(json_file)
        coords = test_j['label_info']['annotations']
        Bbox = []
        for i in range(len(coords)):
            Bbox.append(coords[i]['bbox'])

    for i in range(len(Bbox)):
        crop_img = img.crop((Bbox[i][0],Bbox[i][1],Bbox[i][2],Bbox[i][3]))
        crop_img.save(f'{folder}/{animal}/{i}.jpg')


    return crop_img


### Image Path

In [116]:
train_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/*.jpg')
test_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/*.jpg')
val_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/val/images/*.jpg')

### Label assemble

In [118]:
train_labels = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/labels')
test_labels = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/test/labels')
val_labels = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/val/labels')

In [119]:
org_train_pig_label = glob('/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/*.json')
org_train_cow_label = glob('/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox/*.json')

org_val_pig_label = glob('/content/drive/MyDrive/Innopolis_spark/data/val/label/돼지_bbox/*.json')
org_val_cow_label = glob('/content/drive/MyDrive/Innopolis_spark/data/val/label/소_bbox/*.json')

In [171]:
Validation_pig = os.listdir('/content/drive/MyDrive/Innopolis_spark/data/val/img/돼지_bbox')
len(Validation_pig)

538

In [172]:
Validation_cow = os.listdir('/content/drive/MyDrive/Innopolis_spark/data/val/img/소_bbox')
len(Validation_cow)

1520

In [ ]:
print(len(train_labels))
print(len(test_labels))
print(len(val_labels))

print()

print(len(org_train_pig_label))
print(len(org_train_cow_label))
print(len(org_val_pig_label))
print(len(org_val_cow_label))

#### json_txt

In [122]:
train_labels_json = []
test_labels_json = []
val_labels_json = []
for i in tqdm(train_labels):
    train_labels_json.append(i.replace('txt','json'))

for i in tqdm(test_labels):
    test_labels_json.append(i.replace('txt','json'))
    
for i in tqdm(val_labels):
    val_labels_json.append(i.replace('txt','json'))

100%|██████████| 3291/3291 [00:00<00:00, 1403645.97it/s]


#### Json Result

In [120]:
Train = []
Test = []
Val = []
Error = []

In [ ]:
for i in tqdm(train_labels_json):
    try:
        for j in org_train_pig_label:
            if i in j :
                Train.append(j)

        for k in org_train_cow_label:
            if i in k :
                Train.append(k)
                
        for l in org_val_pig_label:
            if i in l :
                Train.append(l)

        for m in org_val_cow_label:
            if i in m :
                Train.append(m)

    except Exception as e:
        print(e)
        Error.append(i)

In [ ]:
for i in tqdm(test_labels_json):
    try:
        for j in org_train_pig_label:
            if i in j :
                Test.append(j)

        for k in org_train_cow_label:
            if i in k :
                Test.append(k)
                
        for l in org_val_pig_label:
            if i in l :
                Test.append(l)

        for m in org_val_cow_label:
            if i in m :
                Test.append(m)

    except Exception as e:
        print(e)
        Error.append(i)

In [123]:
for i in tqdm(val_labels_json):
    try:
        for j in org_train_pig_label:
            if i in j :
                Val.append(j)

        for k in org_train_cow_label:
            if i in k :
                Val.append(k)
                
        for l in org_val_pig_label:
            if i in l :
                Val.append(l)

        for m in org_val_cow_label:
            if i in m :
                Val.append(m)

    except Exception as e:
        print(e)
        Error.append(i)

100%|██████████| 3291/3291 [00:09<00:00, 343.38it/s]


In [124]:
print(len(Train))
print(len(Test))
print(len(Val))

0
0
3291


In [ ]:
print(len(train_labels))
print(len(test_labels))
print(len(val_labels))

In [ ]:
Train = sorted(Train)
Test = sorted(Test)
Val = sorted(Val)

train_labels_json = sorted(train_labels_json)
test_labels_json = sorted(test_labels_json)
val_labels_json = sorted(val_labels_json)

train_imgs = sorted(train_imgs)
test_imgs = sorted(test_imgs)
val_imgs = sorted(val_imgs)

In [ ]:
train_imgs[0]

In [ ]:
Train[0]

In [142]:
%cd /content/drive/MyDrive/Innopolis_spark/팡훈

/content/drive/MyDrive/Innopolis_spark/팡훈


In [144]:
!mkdir './label'
!mkdir './label/train'
!mkdir './label/test'
!mkdir './label/val'

mkdir: cannot create directory ‘./label’: File exists
mkdir: cannot create directory ‘./label/train’: File exists
mkdir: cannot create directory ‘./label/test’: File exists


In [ ]:
Train = sorted(Train)
Test = sorted(Test)
Val = sorted(Val)

In [ ]:
Train[0]

In [136]:
train_dsc = '/content/drive/MyDrive/Innopolis_spark/팡훈/label/train'
test_dsc = '/content/drive/MyDrive/Innopolis_spark/팡훈/label/test'
val_dsc = '/content/drive/MyDrive/Innopolis_spark/팡훈/label/val'

In [ ]:
for i in tqdm(Train):
    try:
        shutil.copy(i,train_dsc)
    except FileNotFoundError:
        shutil.copy(i.replace('train','val',train_dsc))

In [ ]:
for i in tqdm(Test):
    try:
        shutil.copy(i,test_dsc)
    except:
        shutil.copy(i.replace('train','val',test_dsc))

In [145]:
for i in tqdm(Val):
    try:
        shutil.copy(i,val_dsc)
    except:
        shutil.copy(i.replace('train','val',val_dsc))

100%|██████████| 3291/3291 [00:19<00:00, 172.19it/s]


In [146]:
len(os.listdir(val_dsc))

3291

# 순서확인

###### Train

In [193]:
train_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images')
train_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/train')

print(len(train_imgs))
print(len(train_labs))

train_imgs = sorted(train_imgs)
train_labs = sorted(train_labs)


check_imgs = []
for img in train_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in train_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in train_labs:
    if lab.replace('json','jpg') in train_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in train_imgs:
    if imgs.replace('jpg','json') in train_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)

13163
13163
livestock_cow_bbox_000001
livestock_cow_bbox_000001
False
livestock_cow_bbox_008745.json
livestock_cow_bbox_008163.jpg


###### Test

In [168]:
test_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images')
test_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/test')

print(len(test_imgs))
print(len(test_labs))

test_imgs = sorted(test_imgs)
test_labs = sorted(test_labs)


check_imgs = []
for img in test_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in test_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in test_labs:
    if lab.replace('json','jpg') in test_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in test_imgs:
    if imgs.replace('jpg','json') in test_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)

2058
2058
livestock_cow_bbox_006077
livestock_cow_bbox_006077
True


###### Val

In [170]:
val_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/val/images')
val_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/val')

print(len(val_imgs))
print(len(val_labs))

val_imgs = sorted(val_imgs)
val_labs = sorted(val_labs)


check_imgs = []
for img in val_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in val_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in val_labs:
    if lab.replace('json','jpg') in val_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in val_imgs:
    if imgs.replace('jpg','json') in val_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)

3291
3291
livestock_cow_bbox_011243
livestock_cow_bbox_011243
True


### change

In [173]:
# livestock_cow_bbox_008745.json 
# livestock_cow_bbox_008163.jpg 

In [188]:
train_cow_label_change = os.listdir('/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox')
train_cow_image_change = os.listdir('/content/drive/MyDrive/Innopolis_spark/data/train/img/소_bbox')

In [189]:
print('livestock_cow_bbox_008745.json' in train_labs) # 있음
print('livestock_cow_bbox_008745.jpg' in train_imgs) # 없음  --> (raw -> copy)

print('livestock_cow_bbox_008163.json' in train_labs) # 없음 --> (raw -> copy)
print('livestock_cow_bbox_008163.jpg' in train_imgs) # 있음

print('livestock_cow_bbox_008745.jpg' in train_cow_image_change) # 원본에도 없음 -- > json 제거
print('livestock_cow_bbox_008163.json' in train_cow_label_change)

True
False
False
True
False
True


In [194]:
train_labs.remove('livestock_cow_bbox_008745.json')

In [195]:
len(train_labs)

13162

In [196]:
print('livestock_cow_bbox_008745.json' in train_labs) # 있음

False


In [197]:
with open('/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox/livestock_cow_bbox_008163.json','r') as json_file:
    A = json.load(json_file)
print(A)

{'label_info': {'image': {'file_name': 'livestock_cow_bbox_008163.jpg', 'width': 1920, 'height': 1080, 'farm_name': 'Yangpyeong', 'farm_env': 'open', 'time': 'daytime'}, 'annotations': [], 'categories': [{'supercategory': 'livestock_cow', 'id': 1, 'name': 'cattle'}, {'supercategory': 'livestock_cow', 'id': 2, 'name': 'nuptial'}, {'supercategory': 'livestock_cow', 'id': 3, 'name': 'calf'}]}}


In [198]:
shutil.copy('/content/drive/MyDrive/Innopolis_spark/data/train/label/소_bbox/livestock_cow_bbox_008163.json',
            '/content/drive/MyDrive/Innopolis_spark/팡훈/label/train/livestock_cow_bbox_008163.json')

'/content/drive/MyDrive/Innopolis_spark/팡훈/label/train/livestock_cow_bbox_008163.json'

In [200]:
!rm '/content/drive/MyDrive/Innopolis_spark/팡훈/label/train/livestock_cow_bbox_008745.json'

In [201]:
train_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images')
train_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/train')

print(len(train_imgs))
print(len(train_labs))

train_imgs = sorted(train_imgs)
train_labs = sorted(train_labs)


check_imgs = []
for img in train_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in train_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in train_labs:
    if lab.replace('json','jpg') in train_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in train_imgs:
    if imgs.replace('jpg','json') in train_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)

13163
13163
livestock_cow_bbox_000001
livestock_cow_bbox_000001
True


# [런타임 끊기면 여기서부터]

In [243]:
import os
from glob import glob
import json
from tqdm import tqdm

import shutil

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
import cv2

# Re check

In [216]:
train_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images')
train_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/train')

print(len(train_imgs))
print(len(train_labs))

train_imgs = sorted(train_imgs)
train_labs = sorted(train_labs)


check_imgs = []
for img in train_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in train_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in train_labs:
    if lab.replace('json','jpg') in train_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in train_imgs:
    if imgs.replace('jpg','json') in train_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)



print(train_imgs[8000])
print(train_labs[8000])

13163
13163
livestock_cow_bbox_000001
livestock_cow_bbox_000001
True
livestock_cow_bbox_009522.jpg
livestock_cow_bbox_009522.json


In [217]:
test_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images')
test_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/test')

print(len(test_imgs))
print(len(test_labs))

test_imgs = sorted(test_imgs)
test_labs = sorted(test_labs)


check_imgs = []
for img in test_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in test_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in test_labs:
    if lab.replace('json','jpg') in test_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in test_imgs:
    if imgs.replace('jpg','json') in test_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)


print(test_imgs[1150])
print(test_labs[1150])

2058
2058
livestock_cow_bbox_006077
livestock_cow_bbox_006077
True
livestock_cow_bbox_014063.jpg
livestock_cow_bbox_014063.json


In [218]:
val_imgs = os.listdir('/content/drive/MyDrive/Innopolis_spark/chl/train/data/val/images')
val_labs = os.listdir('/content/drive/MyDrive/Innopolis_spark/팡훈/label/val')

print(len(val_imgs))
print(len(val_labs))

val_imgs = sorted(val_imgs)
val_labs = sorted(val_labs)


check_imgs = []
for img in val_imgs:
    check_imgs.append(img.split('.jpg')[0])


check_label = []
for label in val_labs:
    check_label.append(label.split('.json')[0])


check_imgs = sorted(check_imgs)
check_label = sorted(check_label)

print(check_imgs[0])
print(check_label[0])

print(check_imgs==check_label)

error = []
for lab in val_labs:
    if lab.replace('json','jpg') in val_imgs:
        pass
    else:
        error.append(lab)
        print(lab)

error_img = []
for imgs in val_imgs:
    if imgs.replace('jpg','json') in val_labs:
        pass
    else:
        error_img.append(imgs)
        print(imgs)


print(val_imgs[1150])
print(val_labs[1150])

3291
3291
livestock_cow_bbox_011243
livestock_cow_bbox_011243
True
livestock_cow_bbox_012393.jpg
livestock_cow_bbox_012393.json


# Crop

In [211]:
#초기화
#shutil.rmtree('/content/individual_images')

In [212]:
!mkdir '/content/individual_images'
!mkdir '/content/individual_images/train'
!mkdir '/content/individual_images/test'
!mkdir '/content/individual_images/val'

!mkdir '/content/individual_images/train/cow'
!mkdir '/content/individual_images/test/cow'
!mkdir '/content/individual_images/val/cow'

!mkdir '/content/individual_images/train/pig'
!mkdir '/content/individual_images/test/pig'
!mkdir '/content/individual_images/val/pig'


# extraction_crop FUNCTION

In [221]:
def extraction_crop(image,label,folder_path,Error_imgs,Error_labs):
    for number,(im,lab) in enumerate(zip(image,label)):
        try:     
            img = Image.open(im)

            if 'cow' in im and lab:
                animal = 'cow'
            elif 'pig' in im and lab:
                animal = 'pig'

            fname = im.split('images/')[1].split('.jpg')[0]


            with open(lab,'r') as json_file:
                test_j = json.load(json_file)
                coords = test_j['label_info']['annotations']
                Bbox = []
                for i in range(len(coords)):
                    Bbox.append(coords[i]['bbox'])

            for i in range(len(Bbox)):
                crop_img = img.crop((Bbox[i][0],Bbox[i][1],Bbox[i][2],Bbox[i][3]))
                result = crop_img.resize((256,256))
                result.save(f'{folder_path}/{animal}/{fname}_{i}.jpg')

            print(f'{number} >>  {result.size} >>  {im}        >>>>>          [완료]')

        except Exception as e:
            print()
            print(' <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ERROR >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ')
            print(e)
            Error_imgs.append(im)
            Error_labs.append(lab)
            print(im)
            print(lab)
            print(' <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ERROR >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ')



 

### Test_TEST

In [222]:
train_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/*.jpg')
train_labs = glob('/content/drive/MyDrive/Innopolis_spark/팡훈/label/train/*.json')

train_imgs = sorted(train_imgs)
train_labs = sorted(train_labs)

In [236]:
A_imgs = train_imgs[:10]
A_labs = train_labs[:10]

B_imgs = train_imgs[-10:]
B_labs = train_labs[-10:]

In [237]:
shutil.rmtree('/content/A')

In [239]:
!mkdir '/content/A'
!mkdir '/content/A/train'
!mkdir '/content/A/test'
!mkdir '/content/A/val'

!mkdir '/content/A/train/cow'
!mkdir '/content/A/test/cow'
!mkdir '/content/A/val/cow'

!mkdir '/content/A/train/pig'
!mkdir '/content/A/test/pig'
!mkdir '/content/A/val/pig'


In [240]:
test_train_path = '/content/A/train'

In [241]:
test_Error_imgs = []
test_Error_labs = []
extraction_crop(A_imgs,A_labs,test_train_path,test_Error_imgs,test_Error_labs)
extraction_crop(B_imgs,B_labs,test_train_path,test_Error_imgs,test_Error_labs)

0 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000001.jpg        >>>>>          [완료]
1 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000002.jpg        >>>>>          [완료]
2 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000003.jpg        >>>>>          [완료]
3 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000004.jpg        >>>>>          [완료]
4 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000005.jpg        >>>>>          [완료]
5 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000006.jpg        >>>>>          [완료]
6 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000007.jpg        >>>>>

##### Train

In [242]:
train_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/*.jpg')
train_labs = glob('/content/drive/MyDrive/Innopolis_spark/팡훈/label/train/*.json')

train_imgs = sorted(train_imgs)
train_labs = sorted(train_labs)

In [220]:
print(train_imgs[0])
print(train_labs[0])

/content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_000001.jpg
/content/drive/MyDrive/Innopolis_spark/팡훈/label/train/livestock_cow_bbox_000001.json


In [110]:
print(len(train_imgs))
print(len(train_labs))

13163
13163


In [84]:
Error_imgs = []
Error_labs = []
extraction_crop(train_imgs,train_labs,'/content/individual_images/train',Error_imgs,Error_labs)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_009685.jpg               >>>>>               [완료]
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_009686.jpg               >>>>>               [완료]
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_009687.jpg               >>>>>               [완료]
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_009688.jpg               >>>>>               [완료]
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_009689.jpg               >>>>>               [완료]
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/train/images/livestock_cow_bbox_009690.jpg               >>>>>               [완료]
(256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/

In [88]:
train_cow = os.listdir('/content/individual_images/train/cow')
len(train_cow)

32156

In [89]:
train_pig = os.listdir('/content/individual_images/train/pig')
len(train_pig)

37446

In [92]:
test_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/*.jpg')
test_labs = glob('/content/drive/MyDrive/Innopolis_spark/팡훈/label/test/*.json')

test_imgs = sorted(test_imgs)
test_labs = sorted(test_labs)

In [94]:
print(len(test_imgs))
print(len(test_labs))

2058
2058


In [107]:
Error_imgs_test = []
Error_labs_test = []
extraction_crop(test_imgs,test_labs,'/content/individual_images/test',Error_imgs_test,Error_labs_test)

0 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006077.jpg        >>>>>          [완료]
1 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006078.jpg        >>>>>          [완료]
2 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006079.jpg        >>>>>          [완료]
3 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006080.jpg        >>>>>          [완료]
4 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006081.jpg        >>>>>          [완료]
5 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006082.jpg        >>>>>          [완료]
6 >>  (256, 256) >>  /content/drive/MyDrive/Innopolis_spark/chl/train/data/test/images/livestock_cow_bbox_006083.jpg        >>>>>       

In [127]:

val_imgs = glob('/content/drive/MyDrive/Innopolis_spark/chl/train/data/val/images/*.jpg')
# val_labs = glob('/content/drive/MyDrive/Innopolis_spark/팡훈/label/val/*.json')

val_imgs = sorted(val_imgs)
Val = sorted(Val)

In [128]:
print(len(val_imgs))
print(len(Val))

3291
3291


In [114]:
val_imgs[0]

'/content/drive/MyDrive/Innopolis_spark/chl/train/data/val/images/livestock_cow_bbox_011243.jpg'

In [129]:
Val[0]

'/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003981.json'

In [130]:
val_labs[0]

'/content/drive/MyDrive/Innopolis_spark/팡훈/label/val/livestock_cow_bbox_006077.json'

In [131]:
Val

['/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003981.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003982.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003983.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003984.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003985.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003986.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003987.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003988.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003989.json',
 '/content/drive/MyDrive/Innopolis_spark/data/train/label/돼지_bbox/livestock_pig_bbox_003990.json',
 '/content